In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from dataclasses import dataclass

@dataclass
class ColourContext:
    favourite_colour: str = "blue"
    least_favourite_colour: str = "yellow"

In [3]:
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", temperature=0.0)
agent = create_agent(
    model=model,
    context_schema=ColourContext  
)

In [4]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favorite color?")]},
    context=ColourContext()
)

In [5]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is my favorite color?', additional_kwargs={}, response_metadata={}, id='fd1a42db-2387-4982-bd49-a72a9dad3215'),
              AIMessage(content=[], additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bfaba-a4b0-7361-918b-2e677d94b1cc-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 7, 'output_tokens': 373, 'total_tokens': 380, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 373}})]}


Context is passed to the tools directly, not the model.

## Accessing Context

In [6]:
from langchain.tools import tool, ToolRuntime

@tool
def get_favourite_colour(runtime: ToolRuntime) -> str:
    """Get the favourite colour of the user"""
    return runtime.context.favourite_colour

@tool
def get_least_favourite_colour(runtime: ToolRuntime) -> str:
    """Get the least favourite colour of the user"""
    return runtime.context.least_favourite_colour

In [7]:
agent = create_agent(
    model=model,
    tools=[get_favourite_colour, get_least_favourite_colour],
    context_schema=ColourContext
)

In [8]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext()
)

pprint(response)

/Users/lunit_haseebraja/Developer/personal/projects/LLM/langchain-foundations/.venv/lib/python3.13/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=ColourContext(favourite_c...vourite_colour='yellow'), input_type=ColourContext])
  return self.__pydantic_serializer__.to_python(


{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='dd5ad33c-4b16-463a-a234-63555c224c96'),
              AIMessage(content=[], additional_kwargs={'function_call': {'name': 'get_favourite_colour', 'arguments': '{}'}, '__gemini_function_call_thought_signatures__': {'5e1063f9-2c70-42f1-8465-f25d1d488895': 'Ev8CCvwCAXLI2ny2MoBwsh8Wiw+/p7Io+1Lm5BXSnIvgmi8XXnJ77ncg35L9mDEpOM9l4HgmJvTcGZG8DQU8QhxbM+MYMxu/f4Leg0acEYHi7KjXP95GbEu9WzXtmG4tTZcNc0F6d7ZcpSP/0tgRkj2kUMa20QxH/J0xjSzVmWUfthQzJD3aKG1SRCmpSv59UZp7wKbwLgrAV33bnpgF0LKjEfIRyVOi/uh6Uo1Am77Ui1cwc0W7LJkEqBjQsyq7OH3QljxExgQexRThfwedfD4xmIMtH2vt4y3DrA7C/kcxVkfSWaY/LGop3k0d0hM80sRZUT9VTkF4e+9a2RANvvc9ir3cdmCyQ4GqnbAxUC1UeY1pKTNV+jrIdwHS8gKrbnvVSc9nvCVFvhFTxYzDJod+J6z4Hu+Rf/b9zFEeWE/6SuyWiHn0fygNljJ9Hzz35Ar8NUqsiaY9rKxlTJ8ZyLBGyxbMD63fA14jZKq+DrIgNkeqzRoSYJgj2+bBz2Vn660='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'mod

In [9]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext(favourite_colour="green")
)

pprint(response)

/Users/lunit_haseebraja/Developer/personal/projects/LLM/langchain-foundations/.venv/lib/python3.13/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=ColourContext(favourite_c...vourite_colour='yellow'), input_type=ColourContext])
  return self.__pydantic_serializer__.to_python(


{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='10f66f67-1085-4df1-864d-374cc43c9ead'),
              AIMessage(content=[], additional_kwargs={'function_call': {'name': 'get_favourite_colour', 'arguments': '{}'}, '__gemini_function_call_thought_signatures__': {'a7949365-6a8b-4ede-9423-0186dfff0454': 'Ev8CCvwCAXLI2nwT4CnbyUxWHGL2YV7wx4vdPu4MBfufv7M84h8o65uii7wYYznqmiQAs1KKNJRjexK4rERN/+pxD7Oa8R43ReRd+PeN1Y+FTtwuxRexuHOA9Gc1FSidTV/GRhKepYLfjQAmFj3lzOXjiQ+nt5y/xQJg8ruf8es3rHxWFgx9MqyEgX4Pph3HGp3u4VfUzl3XU2qrNptvOeiMEUTjBXn0AgloGvUVMpghZV31fgUCFOgYx3RSvuONgkS7vZPHC95NuDGIZE4ow/4hbHpCfn0sO4fm9MwqrRtU0diL1NbymZtjvpDrA08ukmkdhgWL+fEI3Wi8HMOOGSjpO85qMzyabWxb7CgeblHhc6TYMe5jtqy+cfZ1xc9pFHQJEJyjrVdRFuv8P4OTpeQJdb0GBs4RKXkaD/AOCw/F23mJhRPLHPrXKre9M0omRK9UeWjqQik71xwy5yTGVCblxUntbFUx7q7sN88rzW1iEjWC5AAKR6rTp7GlZ253G50='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'mod